In [22]:
import pandas as pd
import numpy as np
import os
from sklearn import preprocessing  
pd.set_option("display.max_rows", 200)

In [23]:
df=pd.read_csv('test.csv')

In [24]:
X=df.drop(['PassengerId','Name'],axis=1)


In [25]:
X=X.drop('Cabin',axis=1)

In [26]:
X['HomePlanet'].fillna('Earth', inplace=True)
X['HomePlanet'].value_counts()

HomePlanet
Earth     2350
Europa    1002
Mars       925
Name: count, dtype: int64

In [27]:
X['CryoSleep'].fillna(False, inplace=True)
X['CryoSleep'].value_counts()

CryoSleep
False    2733
True     1544
Name: count, dtype: int64

In [28]:
X['Destination'].fillna('TRAPPIST-1e', inplace=True)
X['Destination'].value_counts()

Destination
TRAPPIST-1e      3048
55 Cancri e       841
PSO J318.5-22     388
Name: count, dtype: int64

In [29]:
X['VIP'].fillna(False,inplace=True)
X['VIP'].value_counts()

VIP
False    4203
True       74
Name: count, dtype: int64

In [30]:
X['Age'].fillna(X['Age'].mean(), inplace=True)
X['Age'].isnull().sum()

0

In [31]:
columns_to_fill_zero = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
X[columns_to_fill_zero] = X[columns_to_fill_zero].fillna(0)


In [32]:
# 'HomePlanet'와 'Destination' 열을 범주형으로 변환
X['HomePlanet'] = X['HomePlanet'].astype('category')
X['Destination'] = X['Destination'].astype('category')

# one-hot encoding 수행
X_encoded = pd.get_dummies(X, columns=['HomePlanet', 'Destination'], drop_first=True)


In [33]:
# bool 자료형 열에 대해 True는 1, False는 0으로 변환
bool_columns = ['CryoSleep', 'VIP', 'HomePlanet_Europa', 'HomePlanet_Mars', 'Destination_PSO J318.5-22', 'Destination_TRAPPIST-1e']
X_encoded[bool_columns] = X_encoded[bool_columns].astype(int)


In [34]:
X_encoded['total'] = X_encoded[['RoomService', 'Spa', 'VRDeck', 'FoodCourt', 'ShoppingMall']].sum(axis=1)

In [35]:
X_encoded.head().T

,0,1,2,3,4
CryoSleep,1.0,0.0,1.0,0.0,0.0
Age,27.0,19.0,31.0,38.0,20.0
VIP,0.0,0.0,0.0,0.0,0.0
RoomService,0.0,0.0,0.0,0.0,10.0
FoodCourt,0.0,9.0,0.0,6652.0,0.0
ShoppingMall,0.0,0.0,0.0,0.0,635.0
Spa,0.0,2823.0,0.0,181.0,0.0
VRDeck,0.0,0.0,0.0,585.0,0.0
HomePlanet_Europa,0.0,0.0,1.0,1.0,0.0
HomePlanet_Mars,0.0,0.0,0.0,0.0,0.0


In [36]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()

# # X_encoded를 표준화
# X_encoded_scaled = scaler.fit_transform(X_encoded)

In [37]:
import joblib
scaler = joblib.load('new_scaler.pkl')
X_encoded_scaled = scaler.transform(X_encoded)

In [38]:
X_encoded_scaled[:][5].max()

0.7310690116807856

In [39]:
X_encoded_scaled.shape

(4277, 13)

In [40]:
import joblib

# Load the saved model
loaded_model = joblib.load('mlpv2.pkl')

# 예측할 데이터 준비 (X_encoded_scaled를 사용한다고 가정)
# Replace X_encoded_scaled with your new data
new_data = X_encoded_scaled

# Make predictions
predictions = loaded_model.predict(new_data)

# Display the predictions
print(predictions)


Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2024-02-27 19:45:32         2706
metadata.json                                  2024-02-27 19:45:32           64
variables.h5                                   2024-02-27 19:45:32       571832
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
......dense_3
.........vars
............0
............1
......dropout
.........vars
......dropout_1
.........vars
......dropout_2
.........vars
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
...

In [19]:
import joblib

# Load the saved model
loaded_model = joblib.load('mlpv1.pkl')

# 예측할 데이터 준비 (X_encoded_scaled를 사용한다고 가정)
# Replace X_encoded_scaled with your new data
new_data = X_encoded_scaled

# Make predictions
predictions = loaded_model.predict(new_data)

# Display the predictions
print(predictions)


AttributeError: 'Adam' object has no attribute 'build'

In [41]:
submission_df = pd.read_csv('sample_submission.csv')
submission_df['Transported'] = predictions
submission_df.to_csv('submission_result_randomF.csv', index=False)


In [21]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score, classification_report

# # 랜덤 포레스트 분류기 선택
# model_random_forest = RandomForestClassifier(random_state=42)

# # 모델 학습
# model_random_forest.fit(X_train, y_train)

# # 테스트 데이터에 대한 예측
# y_pred_random_forest = model_random_forest.predict(X_test)

# # 정확도 출력
# accuracy_random_forest = accuracy_score(y_test, y_pred_random_forest)
# print(f'Accuracy with Random Forest classifier: {accuracy_random_forest}')

# # 분류 보고서 출력
# report_random_forest = classification_report(y_test, y_pred_random_forest)
# print('Classification Report with Random Forest classifier:')
# print(report_random_forest)


In [43]:
import pandas as pd

# 파일 로드
df_submission = pd.read_csv('submission_result_randomF.csv')

# 'Transported' 열의 값이 0.5 초과인 경우 TRUE, 그 외에는 FALSE로 변경
df_submission['Transported'] = df_submission['Transported'] > 0.5

# 변경된 데이터프레임을 새로운 CSV 파일로 저장
df_submission.to_csv('updated_submission_result.csv', index=False)
